---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [47]:
import networkx as nx
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [48]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [49]:
def plot(G):
    
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram =[list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
    plt.plot(degree_values,histogram, 'o')
    plt.xlabel('Degree')
    plt.ylabel('Fraction of Nodes')
    plt.xscale('log')
    plt.yscale('log')
    plt.show()
    
def graph_identification():
    
    #for i in P1_Graphs:
    #    plot(i)
    #for i in P1_Graphs:
    #    print(nx.average_shortest_path_length(i),nx.average_clustering(i))
    
    return ['PA','SW_L','SW_L','PA','SW_H']
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [50]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [110]:
def salary_predictions():
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_curve, auc
    from sklearn.preprocessing import MinMaxScaler
    
    df = pd.DataFrame(index=G.nodes())
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness centrality'] = pd.Series(nx.closeness_centrality(G))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    
    df_train = df.where(np.isnan(df['ManagementSalary']) == False).dropna()
    df_test = df.where(np.isnan(df['ManagementSalary']) == True).drop(['ManagementSalary'], axis=1).dropna()
    

    X_train = df_train[['clustering','degree','degree centrality','closeness centrality','betweeness','pr']]
    y_train = df_train['ManagementSalary']

    X_test = df_test[['clustering','degree','degree centrality','closeness centrality','betweeness','pr']]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    lr = LogisticRegression().fit(X_train_scaled, y_train)

    y_scores_lr = lr.decision_function(X_test)
    y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]
    
    
    #fpr_lr, tpr_lr, _ = roc_curve(y_test, y_proba_lr[:, 1])
    #roc_auc_lr = auc(fpr_lr, tpr_lr)
    
    
    return pd.Series(data=y_proba_lr,index=X_test.index)
salary_predictions()

1       0.182372
2       0.460167
5       0.906187
8       0.148184
14      0.325667
18      0.215190
27      0.255241
30      0.278608
31      0.185124
34      0.139532
37      0.138974
40      0.225018
45      0.183901
54      0.209683
55      0.192732
60      0.208589
62      0.959168
65      0.461453
77      0.108798
79      0.159558
97      0.118974
101     0.109317
103     0.204844
108     0.159094
113     0.378859
122     0.101819
141     0.362022
142     0.716541
144     0.092002
145     0.311267
          ...   
913     0.095571
914     0.107660
915     0.034973
918     0.134612
923     0.074344
926     0.100134
931     0.091867
934     0.054784
939     0.052165
944     0.035623
945     0.072957
947     0.109813
950     0.150844
951     0.079107
953     0.077132
959     0.036788
962     0.037156
963     0.159761
968     0.093080
969     0.088206
974     0.089886
984     0.052953
987     0.100618
989     0.093384
991     0.093120
992     0.040781
994     0.037035
996     0.0357

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [86]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [112]:
def new_connections_predictions():
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_curve, auc
    from sklearn.preprocessing import MinMaxScaler

    future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    #future_connections['Common Neighbors'] = [len(list(nx.common_neighbors(G, i[0], i[1]))) for i in future_connections.index ]
    future_connections['Community Common Neighbors'] =[list(nx.cn_soundarajan_hopcroft(G, [(i[0],i[1])],community='Department'))[0][2] for i in future_connections.index ]
    future_connections['jaccard_coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['resource_allocation_index'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    
    df_train = future_connections.where(np.isnan(future_connections['Future Connection']) == False).dropna()
    
    df_test = future_connections.where(np.isnan(future_connections['Future Connection']) == True).drop(['Future Connection'], axis=1).dropna()
    
    X_train = df_train[['preferential attachment','Community Common Neighbors','jaccard_coefficient','resource_allocation_index']]
    y_train = df_train['Future Connection']
    

    X_test = df_test[['preferential attachment','Community Common Neighbors','jaccard_coefficient','resource_allocation_index']]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    lr = LogisticRegression().fit(X_train_scaled, y_train)

    y_scores_lr = lr.decision_function(X_test)
    y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]
    
    
    #fpr_lr, tpr_lr, _ = roc_curve(y_test, y_proba_lr[:, 1])
    #roc_auc_lr = auc(fpr_lr, tpr_lr)
    
    
    return pd.Series(data=y_proba_lr,index=X_test.index)
new_connections_predictions()

(107, 348)    0.027018
(542, 751)    0.011588
(20, 426)     0.550633
(50, 989)     0.011581
(942, 986)    0.011572
(324, 857)    0.011582
(13, 710)     0.165889
(19, 271)     0.155899
(319, 878)    0.011578
(659, 707)    0.011588
(49, 843)     0.011575
(208, 893)    0.011586
(377, 469)    0.011939
(405, 999)    0.018720
(129, 740)    0.017446
(292, 618)    0.026780
(239, 689)    0.011577
(359, 373)    0.011788
(53, 523)     0.067677
(276, 984)    0.011576
(202, 997)    0.011574
(604, 619)    0.166715
(270, 911)    0.011577
(261, 481)    0.080373
(200, 450)    0.972965
(213, 634)    0.011591
(644, 735)    0.094818
(346, 553)    0.011611
(521, 738)    0.011650
(422, 953)    0.021183
                ...   
(672, 848)    0.011577
(28, 127)     0.894412
(202, 661)    0.011614
(54, 195)     0.999940
(295, 864)    0.011583
(814, 936)    0.011590
(839, 874)    0.011572
(139, 843)    0.011582
(461, 544)    0.011679
(68, 487)     0.011684
(622, 932)    0.011585
(504, 936)    0.019961
(479, 528) 